In [2]:
print('first')

first


In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_json('./sarcasm_news_dataset/Sarcasm_Headlines_Dataset.json', lines = True)

In [5]:
data.tail(10)

,article_link,headline,is_sarcastic
26699,https://www.huffingtonpost.com/entry/leather-l...,"what you should buy your 'basic' friend, accor...",0
26700,https://www.huffingtonpost.com/entry/whats-in-...,what's in your mailbox? tips on what to do whe...,0
26701,https://www.huffingtonpost.com/entry/paul-ryan...,paul ryan is more of a con man than ever,0
26702,https://politics.theonion.com/pentagon-to-with...,pentagon to withhold budget figures out of res...,1
26703,https://www.theonion.com/pope-francis-wearing-...,pope francis wearing sweater vestments he got ...,1
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0
26708,https://www.huffingtonpost.com/entry/gourmet-g...,gourmet gifts for the foodie 2014,0


In [6]:
data.is_sarcastic.value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [7]:
data.drop_duplicates(inplace = True)
data.shape

(26708, 3)

In [8]:
s1 = data['headline'][0]
s1

"former versace store clerk sues over secret 'black code' for minority shoppers"

In [9]:
x = data['headline']
y = data['is_sarcastic']
print(x.tail(10))
print(y.tail(10))

26699    what you should buy your 'basic' friend, accor...
26700    what's in your mailbox? tips on what to do whe...
26701             paul ryan is more of a con man than ever
26702    pentagon to withhold budget figures out of res...
26703    pope francis wearing sweater vestments he got ...
26704                 american politics in moral free-fall
26705                              america's best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: headline, dtype: object
26699    0
26700    0
26701    0
26702    1
26703    1
26704    0
26705    0
26706    0
26707    0
26708    0
Name: is_sarcastic, dtype: int64


# Text cleaning


In [10]:

import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer


In [11]:
lemm= WordNetLemmatizer()

In [12]:
import re
def clean_txt(txt):
    txt = re.sub(r'\d+', '',txt) #remove nos
    token = word_tokenize(txt.lower()) #convert text to tokens
    stop_updated = stopwords.words("english") + list(punctuation)  #remove stopwords and punctuations
    final_word = [lemm.lemmatize(term) for term in token if term not in stop_updated and len(term) > 2] 
    #all terms in tokens are 'lemmatized' ie converted to root word (lemma) preserving its meaning
    #example caring will be lemmatized to care, whereas stemming would lead to car
    new_txt = " ".join(final_word)
    return new_txt

In [13]:
data['clean_text'] = data.headline.apply(clean_txt)
print(data['clean_text'].head(10))

0    former versace store clerk sue secret 'black c...
1    'roseanne revival catch thorny political mood ...
2    mom starting fear son web series closest thing...
3    boehner want wife listen come alternative debt...
4    j.k. rowling wish snape happy birthday magical...
5                                advancing world woman
6               fascinating case eating lab-grown meat
7                     ceo send kid school work company
8     top snake handler leaf sinking huckabee campaign
9        friday morning email inside trump presser age
Name: clean_text, dtype: object


In [154]:
data['headline'][4000]

"trump announces he's a very sad man"

In [160]:
print(data['headline'][4000])
print(data['clean_text'][4000])
print(data['is_sarcastic'][4000])


trump announces he's a very sad man
trump announces sad man
1


In [158]:
print(data['headline'][1728])
print(data['clean_text'][1728])
print(data['is_sarcastic'][1728])


joan moran: how to give yourself the gift of time
joan moran give gift time
0


# TF - IDF matrix

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=400)
X = tfidf_vectorizer.fit_transform(data['clean_text'])
TFIDF = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [15]:
(tfidf_vectorizer.get_feature_names_out())

array(['actually', 'age', 'air', 'already', 'always', 'america',
       'american', 'announces', 'another', 'anti', 'area', 'around',
       'art', 'asks', 'attack', 'award', 'away', 'baby', 'back', 'bad',
       'ban', 'behind', 'bernie', 'best', 'better', 'biden', 'big',
       'bill', 'black', 'body', 'book', 'boy', 'break', 'bush',
       'business', 'buy', 'california', 'call', 'campaign', 'cancer',
       'candidate', 'car', 'care', 'case', 'cat', 'ceo', 'change',
       'chief', 'child', 'christmas', 'city', 'claim', 'class', 'climate',
       'clinton', 'college', 'come', 'coming', 'community', 'company',
       'congress', 'control', 'cop', 'could', 'country', 'couple',
       'court', 'cover', 'crisis', 'cruz', 'cut', 'dad', 'daughter',
       'day', 'dead', 'deal', 'death', 'debate', 'demand', 'democrat',
       'department', 'director', 'doctor', 'dog', 'donald', 'dream',
       'drug', 'dy', 'earth', 'eating', 'election', 'email', 'employee',
       'end', 'enough', 'entir

# classification models

## logistic regression

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x = TFIDF
y = data['is_sarcastic']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)
LRclassifier = LogisticRegression()


In [100]:
X_train

,actually,age,air,already,always,america,american,announces,another,anti,...,without,woman,word,work,worker,working,world,would,year,york
25735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
LRclassifier.fit(X_train, y_train)

LogisticRegression()

In [102]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = LRclassifier.predict(X_train)
accuracy_score(y_train,y_pred)

0.7166622091468307

In [103]:
y_pred = LRclassifier.predict(X_test)
accuracy_score(y_test,y_pred)

0.6983651566204917

In [104]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.82      0.70      0.75      5268
           1       0.55      0.70      0.61      2745

    accuracy                           0.70      8013
   macro avg       0.68      0.70      0.68      8013
weighted avg       0.72      0.70      0.71      8013



# SVM

## C = 10

In [105]:
from sklearn.svm import SVC
SVMclassifier = SVC(kernel = 'rbf', C = 10, gamma = 'scale')
SVMclassifier.fit(X_train, y_train)

SVC(C=10)

In [106]:
y_pred = SVMclassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.8878309708478203

In [107]:

y_pred = SVMclassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6796455759390989

In [108]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.78      0.69      0.73      5118
           1       0.55      0.67      0.60      2895

    accuracy                           0.68      8013
   macro avg       0.67      0.68      0.67      8013
weighted avg       0.70      0.68      0.68      8013



## C = 100

In [109]:
from sklearn.svm import SVC
SVMclassifier = SVC(kernel = 'rbf', C = 100, gamma = 'scale')
SVMclassifier.fit(X_train, y_train)

SVC(C=100)

In [110]:
y_pred = SVMclassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.8880984220379781

In [111]:

y_pred = SVMclassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6792711843254711

In [112]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.78      0.69      0.73      5115
           1       0.55      0.66      0.60      2898

    accuracy                           0.68      8013
   macro avg       0.67      0.68      0.67      8013
weighted avg       0.70      0.68      0.68      8013



# MLP

## with logistic activation function

In [149]:
from sklearn.neural_network import MLPClassifier
MLPclassfier = MLPClassifier(activation = 'logistic', hidden_layer_sizes=(200), max_iter = 2000)
MLPclassfier.fit(X_train, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=200, max_iter=2000)

In [114]:

y_pred = MLPclassfier.predict(X_train)
accuracy_score(y_train, y_pred)

0.7163412677186414

In [115]:
y_pred = MLPclassfier.predict(X_test)
accuracy_score(y_test, y_pred)

0.7006115063022589

In [116]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.80      0.70      0.75      5116
           1       0.57      0.69      0.63      2897

    accuracy                           0.70      8013
   macro avg       0.69      0.70      0.69      8013
weighted avg       0.72      0.70      0.71      8013



## with relu activation function

In [117]:
from sklearn.neural_network import MLPClassifier
MLPclassfier = MLPClassifier(activation = 'relu', hidden_layer_sizes=(200), max_iter = 2000)
MLPclassfier.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=200, max_iter=2000)

In [118]:
y_pred = MLPclassfier.predict(X_train)
accuracy_score(y_train, y_pred)

0.866381385397165

In [119]:
y_pred = MLPclassfier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6627979533258455

In [120]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.63      0.73      0.68      3895
           1       0.70      0.60      0.65      4118

    accuracy                           0.66      8013
   macro avg       0.67      0.66      0.66      8013
weighted avg       0.67      0.66      0.66      8013



# KNN

## 3 nearest neighbours

In [121]:
from sklearn.neighbors import KNeighborsClassifier
knnClassfier = KNeighborsClassifier(n_neighbors = 3, weights = 'uniform')
knnClassfier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [122]:
y_pred = knnClassfier.predict(X_train)
accuracy_score(y_train, y_pred)

0.7515913345814389

In [123]:
y_pred = knnClassfier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6111319106452016

In [124]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.64      0.66      0.65      4337
           1       0.58      0.56      0.57      3676

    accuracy                           0.61      8013
   macro avg       0.61      0.61      0.61      8013
weighted avg       0.61      0.61      0.61      8013



## 1 nearest neighbours

In [125]:
from sklearn.neighbors import KNeighborsClassifier
knnClassfier = KNeighborsClassifier(n_neighbors = 1, weights = 'uniform')
#uniform means weights are independent of distance
knnClassfier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [126]:
y_pred = knnClassfier.predict(X_train)
accuracy_score(y_train, y_pred)

0.8709280556298475

In [127]:
y_pred = knnClassfier.predict(X_test)
accuracy_score(y_test, y_pred)


0.6133782603269686

In [128]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.71      0.64      0.67      4971
           1       0.49      0.57      0.53      3042

    accuracy                           0.61      8013
   macro avg       0.60      0.60      0.60      8013
weighted avg       0.63      0.61      0.62      8013



# Naive Bayes

In [129]:

from sklearn.naive_bayes import GaussianNB
NBClassifier = GaussianNB()
NBClassifier.fit(X_train, y_train)

GaussianNB()

In [130]:
y_pred = NBClassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.6701791922974057

In [131]:
y_pred = NBClassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6541869462124048

In [132]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.59      0.74      0.66      3602
           1       0.73      0.59      0.65      4411

    accuracy                           0.65      8013
   macro avg       0.66      0.66      0.65      8013
weighted avg       0.67      0.65      0.65      8013



# Decision Trees

## entropy

In [133]:

from sklearn.tree import DecisionTreeClassifier
DTClassifier = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)
DTClassifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [134]:
y_pred = DTClassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.6549879646964429

In [135]:
y_pred = DTClassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6425808061899414

In [136]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.96      0.62      0.75      6971
           1       0.24      0.82      0.37      1042

    accuracy                           0.64      8013
   macro avg       0.60      0.72      0.56      8013
weighted avg       0.86      0.64      0.70      8013



## gini 

In [137]:
DTClassifier = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)
DTClassifier.fit(X_train, y_train)
pred = DTClassifier.predict(X_test)

In [138]:
y_pred = DTClassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.6549879646964429

In [139]:
y_pred = DTClassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.642705603394484

In [140]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.96      0.62      0.75      6978
           1       0.24      0.82      0.37      1035

    accuracy                           0.64      8013
   macro avg       0.60      0.72      0.56      8013
weighted avg       0.87      0.64      0.70      8013



# Random Forests

In [141]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier(n_estimators = 1000, max_features = 'log2', max_depth = 10)
RFClassifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=1000)

In [142]:
y_pred = RFClassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.6494784701791922

In [143]:
y_pred = RFClassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6347185823037564

In [144]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.61      0.75      7248
           1       0.19      0.89      0.32       765

    accuracy                           0.63      8013
   macro avg       0.59      0.75      0.53      8013
weighted avg       0.91      0.63      0.71      8013



#  AdaBoost

In [145]:
from sklearn.ensemble import AdaBoostClassifier
ABClassifier = AdaBoostClassifier(n_estimators = 20)
ABClassifier.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=20)

In [146]:
y_pred = ABClassifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.6647766782562182

In [147]:
y_pred = ABClassifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6561837014850868

In [148]:

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.94      0.63      0.75      6722
           1       0.29      0.80      0.43      1291

    accuracy                           0.66      8013
   macro avg       0.62      0.71      0.59      8013
weighted avg       0.84      0.66      0.70      8013

